<h1>Affordability</h1>

Measuring affordability of postcode based on median income, median mortage, median rental price and average facilities

In [65]:
# importing necessary libraries
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../scripts/')
from helper_functions import convert_census_to_postcode

In [66]:
# Reading in median rental price per postcode
median_rent_df = pd.read_csv("../data/curated/median_rental_postcode.csv")
median_rent_df.head()

,Postcode,Cost
0,3000,510.0
1,3002,642.5
2,3003,520.0
3,3004,550.0
4,3006,551.5


In [67]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/properties_processed.csv")

# Remove unecessary column "Unnamed: 0"
property_df = property_df.drop(columns = ["Unnamed: 0"])
property_df.head()

,Name,Cost,Coordinates,Bed,Bath,Parking,Property_Type,Agency,Postcode
0,5408/500 Elizabeth Street Melbourne VIC 3000,440.0,"[-37.8072443, 144.9602814]",1,1,0,Apartment / Unit / Flat,BRADY residential,3000
1,502/118 Russell Street Melbourne VIC 3000,620.0,"[-37.8135864, 144.9687232]",1,1,0,Apartment / Unit / Flat,Dingle Partners,3000
2,202A/441 Lonsdale Street Melbourne VIC 3000,300.0,"[-37.8134292, 144.9594445]",1,1,0,Apartment / Unit / Flat,Biggin & Scott Stonnington,3000
3,57/243 Collins Street Melbourne VIC 3000,400.0,"[-37.8159969, 144.9657956]",1,1,0,Apartment / Unit / Flat,Harcourts Melbourne City,3000
4,2311/601 Little Lonsdale Street Melbourne VIC ...,625.0,"[-37.8137564, 144.9537143]",2,2,1,Apartment / Unit / Flat,Harcourts Melbourne City,3000


In [68]:
# Calculate the average number of facilities per postcode
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.579310
3002    1.909091
3003    1.796875
3004    1.763441
3006    1.785714
Name: Bed, dtype: float64

In [69]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.296552
3002    1.318182
3003    1.343750
3004    1.494624
3006    1.469388
Name: Bath, dtype: float64

In [70]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.365517
3002    1.000000
3003    0.609375
3004    1.096774
3006    0.647959
Name: Parking, dtype: float64

In [80]:
# Create dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.579310,1.296552,0.365517
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.763441,1.494624,1.096774
4,3006,1.785714,1.469388,0.647959


In [81]:
# Joining median rental price per suburb with average facilities
facilities_df = facilities_df.set_index("Postcode")
price_by_facility_df = median_rent_df.join(facilities_df, on = "Postcode")
price_by_facility_df = price_by_facility_df.set_index("Postcode")
price_by_facility_df.head()

,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,
3000,510.0,1.579310,1.296552,0.365517
3002,642.5,1.909091,1.318182,1.000000
3003,520.0,1.796875,1.343750,0.609375
3004,550.0,1.763441,1.494624,1.096774
3006,551.5,1.785714,1.469388,0.647959


In [204]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [245]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [246]:
# Remove unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Select columns required for assessing affordability
census_income_df = census_by_postcode_df[["postcode_2021", "avg_med_mortg_rep_21", "avg_med_person_inc_21"]]
census_income_df.head()

,postcode_2021,avg_med_mortg_rep_21,avg_med_person_inc_21
0,3000,2040.19,6467.76
1,3002,2155.22,10432.90
2,3003,2085.00,1000.00
3,3004,2149.75,8339.46
4,3006,2079.00,19507.00


In [247]:
# Renaming postcode column of census dataframe and setting to index
census_income_df = census_income_df.rename({"postcode_2021": "Postcode"}, axis = 1)
census_income_df = census_income_df.set_index("Postcode")
census_income_df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21
Postcode,,
3000,2040.19,6467.76
3002,2155.22,10432.90
3003,2085.00,1000.00
3004,2149.75,8339.46
3006,2079.00,19507.00


In [276]:
# Joining census income dataframe to price by facility dataframe
df = census_income_df.join(price_by_facility_df, on = "Postcode", how = "right")
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,
3000,2040.19,6467.76,510.0,1.579310,1.296552,0.365517
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375
3004,2149.75,8339.46,550.0,1.763441,1.494624,1.096774
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959


Calculating affordability metric

In [277]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [278]:
# Calculating weights for facilities (from previous correlation matrix know positive correlation exists between all 3 attributes and the cost)
bed_weight = df["Average # Beds"].corr(df["Cost"])
bath_weight = df["Average # Baths"].corr(df["Cost"])
parking_weight = df["Average # Parking"].corr(df["Cost"])

In [279]:
# Taking the median rental price divided by the median income (multiply by 100 to get cost as a percentage of income)
df["(Cost / Income)%"] = (df["Cost"] / df["avg_med_person_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%
Postcode,,,,,,,
3000,2040.19,6467.76,510.0,1.579310,1.296552,0.365517,7.885265
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000
3004,2149.75,8339.46,550.0,1.763441,1.494624,1.096774,6.595151
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190


In [280]:
# Taking the median mortage divided by the median income (multiply by 100 to get mortage as a percentage of income)
#avg_median_mortage = sum(df["avg_med_mortg_rep_21"]) / len(df)
#print(avg_median_mortage)
#df["Mortage Proportion"] = df["avg_med_mortg_rep_21"] / avg_median_mortage
df["(Mortage / Income)%"] = (df["avg_med_mortg_rep_21"] / df["avg_med_person_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%
Postcode,,,,,,,,
3000,2040.19,6467.76,510.0,1.579310,1.296552,0.365517,7.885265,31.543997
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000,208.500000
3004,2149.75,8339.46,550.0,1.763441,1.494624,1.096774,6.595151,25.778048
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713


In [281]:
# Calculating non-stadardised affordability metric
#df["Affordability"] = df["Mortage Proportion"] + df["# Beds / Cost"] + df["# Baths / Cost"] + df["# Parking / Cost"] + df["Cost / Income"]
df["Affordability"] = ((bed_weight * df["Average # Beds"]) + (bath_weight * df["Average # Baths"]) + (parking_weight * df["Average # Parking"]) + 
                        df["(Cost / Income)%"] + df["(Mortage / Income)%"] + (1 / df["Cost"]))
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability
Postcode,,,,,,,,,
3000,2040.19,6467.76,510.0,1.579310,1.296552,0.365517,7.885265,31.543997,40.692763
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919,28.377626
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000,208.500000,261.929074
3004,2149.75,8339.46,550.0,1.763441,1.494624,1.096774,6.595151,25.778048,33.985671
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713,14.976319


In [285]:
# Calculating standardised affordability metric
min_affordability = df.sort_values(by = "Affordability").head(1)["Affordability"].tolist()[0]
#print(min_affordability)
max_affordability = df.sort_values(by = "Affordability", ascending = False).head(1)["Affordability"].tolist()[0]
#print(max_affordability)

df["Standardised Affordability"] = (df["Affordability"] - min_affordability) / (max_affordability - min_affordability)
df.sort_values(by = "Affordability").head(10)


,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability,Standardised Affordability
Postcode,,,,,,,,,,
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713,14.976319,0.000000
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919,28.377626,0.027973
3004,2149.75,8339.46,550.0,1.763441,1.494624,1.096774,6.595151,25.778048,33.985671,0.039679
3000,2040.19,6467.76,510.0,1.579310,1.296552,0.365517,7.885265,31.543997,40.692763,0.053679
3424,714.33,633.33,115.0,0.500000,1.000000,1.000000,18.157990,112.789541,131.852789,0.243959
3414,737.00,658.00,245.0,2.000000,1.000000,1.000000,37.234043,112.006079,150.694600,0.283288
3418,737.00,658.00,250.0,2.000000,1.000000,2.000000,37.993921,112.006079,151.717177,0.285423
3409,946.00,752.00,200.0,1.000000,1.000000,1.000000,26.595745,125.797872,153.479790,0.289102
3396,714.33,633.33,255.0,2.500000,1.000000,1.000000,40.263370,112.789541,154.691840,0.291632


In [286]:
df.sort_values(by = "Affordability", ascending = False).head(10)

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability,Standardised Affordability
Postcode,,,,,,,,,,
3944,1888.00,751.00,1800.000,3.500000,2.000000,3.000000,239.680426,251.398136,494.058079,1.000000
3929,2244.00,1002.00,2650.000,5.500000,3.000000,2.000000,264.471058,223.952096,492.327328,0.996387
3621,1346.00,692.67,1825.000,3.000000,2.000000,2.000000,263.473227,194.320528,460.325872,0.929590
3150,2500.00,711.33,580.000,3.411765,1.894118,1.835294,81.537402,351.454318,435.586206,0.877950
3564,1205.40,647.00,1495.165,2.727273,1.409091,1.681818,231.091963,186.306028,419.480502,0.844332
3104,2796.50,820.00,620.000,3.224490,1.775510,1.795918,75.609756,341.036585,419.107948,0.843555
3107,2425.00,732.67,625.000,3.470588,2.000000,1.647059,85.304434,330.981206,418.899816,0.843120
3106,2600.00,767.00,557.500,3.409091,2.090909,1.863636,72.685789,338.983051,414.358249,0.833640
3126,3084.00,937.67,775.000,3.148148,1.703704,1.814815,82.651679,328.900359,413.957848,0.832805
